In [ ]:
# SOURCE: https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=overview
# ERA 5 documentation: https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation

In [ ]:
# Import libraries
import cdsapi
import xarray as xr
import rioxarray
from pyproj import Transformer
import os
from datetime import datetime, timedelta
import warnings
import glob
import numpy as np
import pandas as pd

In [ ]:
# Configuration & parameters
WEB_MERCATOR_EXTENT = [
    -10705956.3396000005304813,  # x_min (west)
    290328.920800000021699,      # y_min (south) 
    -6626467.69309999998003244,  # x_max (east)
    2842131.11159999994561108    # y_max (north)
]

# Output directories
OUTPUT_DIR_GRIB = "era5_data"
OUTPUT_DIR_GEOTIFF = "era5_geotiffs"

# Hurricane timeline
DATES = ['2020-10-31', '2020-11-01', '2020-11-02', '2020-11-03', '2020-11-04', '2020-11-05', '2020-11-06', '2020-11-07', '2020-11-08']
TIMES = [f'{h:02d}:00' for h in range(24)]  # Hourly data

# Variables to download
VARIABLES_CONFIG = {
    'core_variables': [
        '10m_u_component_of_wind',
        '10m_v_component_of_wind', 
        'mean_sea_level_pressure',
        'total_precipitation',
        '2m_temperature'
    ]
}

print("Configuration loaded:")
print(f"   Dates: {DATES[0]} to {DATES[-1]}")
print(f"   Variables: {len(VARIABLES_CONFIG['core_variables'])}")

In [ ]:
# Helper functions
def convert_extent_to_latlon(web_mercator_extent):
    """Convert Web Mercator extent to WGS84 lat/lon for ERA5 API"""
    x_min, y_min, x_max, y_max = web_mercator_extent
    
    # Create transformer from Web Mercator to WGS84
    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
    
    # Transform corners
    lon_min, lat_min = transformer.transform(x_min, y_min)
    lon_max, lat_max = transformer.transform(x_max, y_max)
    
    # ERA5 API expects [north, west, south, east]
    return [lat_max, lon_min, lat_min, lon_max]

# Test the conversion
area = convert_extent_to_latlon(WEB_MERCATOR_EXTENT)
print(f"Download area (N,W,S,E): {area}")
print(f"   Coverage: {area[0]:.2f}°N to {area[2]:.2f}°N, {area[1]:.2f}°W to {area[3]:.2f}°W")

In [ ]:
# Create output directories
os.makedirs(OUTPUT_DIR_GRIB, exist_ok=True)
os.makedirs(OUTPUT_DIR_GEOTIFF, exist_ok=True)

print(f"Created directories:")
print(f"   {OUTPUT_DIR_GRIB}/ - for GRIB files")
print(f"   {OUTPUT_DIR_GEOTIFF}/ - for GeoTIFF files")

In [ ]:
# Initialize CDS API client (~/.cdsapirc needs to be configured)
# content of ~/.cdsapirc file: 
# url: https://cds.climate.copernicus.eu/api/v2
# key: YOUR_API_KEY_HERE
try:
    c = cdsapi.Client()
    print("CDS API client initialized successfully!")
    print("   Make sure your ~/.cdsapirc file is configured with your API key")
except Exception as e:
    print(f"Error initializing CDS API: {e}")
    print("   Please check your ~/.cdsapirc configuration")

In [ ]:
# Download variables
def download_variable_set(var_set_name, variables, area, dates, times, output_dir):
    """Download a set of variables from ERA5"""
    filename = f"{output_dir}/era5_{var_set_name}.grib"
    
    print(f"Downloading {var_set_name}...")
    print(f"   Variables: {variables}")
    print(f"   Output: {filename}")
    
    try:
        c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': variables,
                'date': dates,
                'time': times,
                'area': area,  # [north, west, south, east]
                'format': 'grib',
                'grid': [0.25, 0.25],  # 0.25 degree resolution
            },
            filename
        )
        print(f"Downloaded {var_set_name} to {filename}")
        return filename
        
    except Exception as e:
        print(f"Error downloading {var_set_name}: {e}")
        return None

# Download core variables
core_file = download_variable_set(
    'core_variables', 
    VARIABLES_CONFIG['core_variables'], 
    area, 
    DATES, 
    TIMES, 
    OUTPUT_DIR_GRIB
)

In [ ]:
# Convert GRIB to GeoTIFF
def convert_grib_to_geotiff(grib_file, output_dir):
    """Convert GRIB2 files to individual GeoTIFF files"""
    
    if not os.path.exists(grib_file):
        print(f"File not found: {grib_file}")
        return
    
    print(f"Converting {grib_file} to GeoTIFF format...")
    
    try:
        # Open GRIB file with xarray
        ds = xr.open_dataset(grib_file, engine='cfgrib', backend_kwargs={'indexpath': ''})
        
        # Set spatial dimensions
        ds = ds.rename({'longitude': 'x', 'latitude': 'y'})
        ds = ds.rio.set_spatial_dims(x_dim='x', y_dim='y')
        ds = ds.rio.write_crs("EPSG:4326")
        
        # Convert each variable
        converted_files = []
        for var_name in ds.data_vars:
            print(f"   Converting variable: {var_name}")
            
            var_data = ds[var_name]
            
            # Handle different time dimensions
            if 'time' in var_data.dims:
                # Save each time step separately
                for i, time_val in enumerate(var_data.time.values):
                    time_str = str(time_val)[:13].replace(':', '').replace('-', '').replace(' ', '_')
                    
                    output_file = f"{output_dir}/{var_name}_{time_str}.tif"
                    
                    # Select single time step
                    time_slice = var_data.isel(time=i)
                    
                    # Save as GeoTIFF
                    time_slice.rio.to_raster(output_file, compress='lzw')
                    converted_files.append(output_file)
                    
            else:
                # Single time step
                output_file = f"{output_dir}/{var_name}.tif"
                var_data.rio.to_raster(output_file, compress='lzw')
                converted_files.append(output_file)
        
        print(f"Converted {len(converted_files)} files from {grib_file}")
        return converted_files
        
    except Exception as e:
        print(f"Error converting {grib_file}: {e}")
        return []

print("Conversion function ready!")

In [ ]:
# Convert variables
if core_file and os.path.exists(core_file):
    core_tiffs = convert_grib_to_geotiff(core_file, OUTPUT_DIR_GEOTIFF)
else:
    print("Core variables GRIB file not found")

In [ ]:
# Calculate wind speed magnitude
def create_wind_speed_magnitude(output_dir):
    """Calculate wind speed magnitude from u and v components"""
    print("Calculating wind speed magnitude...")
    
    try:
        # Look for the ACTUAL filenames that were created
        u_files = sorted(glob.glob(f"{output_dir}/u10_*.tif"))  # Changed from 10m_u_component_of_wind
        v_files = sorted(glob.glob(f"{output_dir}/v10_*.tif"))  # Changed from 10m_v_component_of_wind
        
        print(f"   Found {len(u_files)} u-component files")
        print(f"   Found {len(v_files)} v-component files")
        
        if len(u_files) == 0:
            print("No u-component files found. Checking available files...")
            all_files = glob.glob(f"{output_dir}/*.tif")
            wind_files = [f for f in all_files if 'u10' in f or 'v10' in f]
            print(f"   Available wind files sample: {wind_files[:3]}")
            return []
        
        wind_speed_files = []
        for u_file, v_file in zip(u_files, v_files):
            # Extract timestamp from filename
            timestamp = u_file.split('_')[-1].replace('.tif', '')
            
            # Open u and v components
            u_data = rioxarray.open_rasterio(u_file)
            v_data = rioxarray.open_rasterio(v_file)
            
            # Calculate wind speed magnitude
            wind_speed = (u_data**2 + v_data**2)**0.5
            
            # Save wind speed
            output_file = f"{output_dir}/wind_speed_{timestamp}.tif"  # Simplified name
            wind_speed.rio.to_raster(output_file, compress='lzw')
            wind_speed_files.append(output_file)
        
        print(f"Created {len(wind_speed_files)} wind speed magnitude files")
        return wind_speed_files
        
    except Exception as e:
        print(f"Error creating wind speed files: {e}")
        return []

# Run the corrected wind speed calculation
wind_speed_files = create_wind_speed_magnitude(OUTPUT_DIR_GEOTIFF)

In [ ]:
# Extract precipitation hours
def extract_all_precipitation_hours(grib_file, output_dir):
    """Extract ALL precipitation time steps (every hour)"""
    
    try:
        ds_precip = xr.open_dataset(
            grib_file, 
            engine='cfgrib',
            backend_kwargs={
                'filter_by_keys': {'shortName': 'tp'},
                'indexpath': ''
            }
        )
        
        var_data = ds_precip['tp']
        print(f"   Full shape: {var_data.shape}")
        print(f"   Dimensions: {var_data.dims}")
        print(f"   Time points: {len(var_data.time)}")
        print(f"   Step points: {len(var_data.step)}")
        
        # Set spatial dimensions
        ds_precip = ds_precip.rename({'longitude': 'x', 'latitude': 'y'})
        ds_precip = ds_precip.rio.set_spatial_dims(x_dim='x', y_dim='y')
        ds_precip = ds_precip.rio.write_crs("EPSG:4326")
        var_data = ds_precip['tp']
        
        precip_files = []
        total_combinations = len(var_data.time) * len(var_data.step)
        
        print(f"   Processing {total_combinations} time-step combinations...")
        
        # Process each time × step combination
        for i, time_val in enumerate(var_data.time.values):
            for j, step_val in enumerate(var_data.step.values):
                
                # Parse time and step
                base_time = pd.to_datetime(time_val)
                step_hours = int(step_val.astype('timedelta64[h]') / pd.Timedelta('1 hour'))
                
                # Calculate actual time (base_time + step)
                actual_time = base_time + pd.Timedelta(hours=step_hours)
                
                # Create timestamp string
                time_str = actual_time.strftime('%Y%m%dT%H')
                
                # Select this specific time and step
                time_step_slice = var_data.isel(time=i, step=j)
                
                # Save as single-band GeoTIFF
                output_file = f"{output_dir}/total_precipitation_{time_str}.tif"
                time_step_slice.rio.to_raster(output_file, compress='lzw')
                precip_files.append(output_file)
                
                # Progress update
                file_num = i * len(var_data.step) + j + 1
                if file_num % 24 == 0 or file_num == total_combinations:
                    print(f"     ✓ Processed {file_num}/{total_combinations} files")
        
        print(f"Extracted {len(precip_files)} hourly precipitation files")
        
        # Remove old files (the 23 incomplete ones)
        old_files = glob.glob(f"{output_dir}/total_precipitation_*T*.tif")
        old_files = [f for f in old_files if f not in precip_files]
        for old_file in old_files:
            try:
                os.remove(old_file)
            except:
                pass
        
        return sorted(precip_files)
        
    except Exception as e:
        print(f"Error: {e}")
        return []

# Run the corrected extraction
all_precip_files = extract_all_precipitation_hours(
    "era5_data/era5_core_variables.grib", 
    OUTPUT_DIR_GEOTIFF
)

In [ ]:
# Check output files
# List all generated files
print("Generated Files Summary:")
print("=" * 50)

# Check GRIB files
grib_files = glob.glob(f"{OUTPUT_DIR_GRIB}/*.grib")
print(f"\nGRIB files ({len(grib_files)}):")
for f in grib_files:
    size_mb = os.path.getsize(f) / (1024*1024)
    print(f"   {os.path.basename(f)} ({size_mb:.1f} MB)")

# Check GeoTIFF files by variable
tiff_files = glob.glob(f"{OUTPUT_DIR_GEOTIFF}/*.tif")
print(f"\nGeoTIFF files ({len(tiff_files)}):")

# Group by variable
variables_found = {}
for f in tiff_files:
    var_name = '_'.join(os.path.basename(f).split('_')[:-1])
    if var_name not in variables_found:
        variables_found[var_name] = 0
    variables_found[var_name] += 1

for var, count in sorted(variables_found.items()):
    print(f"   {var}: {count} time steps")

print(f"\nProcessing complete! Files ready for GIS analysis.")